In [1]:
from numba import cuda
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
im = plt.imread("/content/drive/MyDrive/Colab Notebooks/image.jpg")

In [4]:
shape = np.shape(im)
shape

(1080, 1920, 3)

In [5]:
imflatten = im.copy().reshape(shape[0]*shape[1],3)

In [6]:
shape2 = np.shape(imflatten)

In [7]:
t1 = time.time()

for i in imflatten:
  gray = ((i[0]+i[1]+[2])/3)
  i[0], i[1], i[2] = gray, gray, gray

t2 = time.time()

print(t2-t1)


<ipython-input-7-f808397ba390>:4: RuntimeWarning: overflow encountered in ubyte_scalars
  gray = ((i[0]+i[1]+[2])/3)


20.037894248962402


In [8]:
imgray1 = imflatten.reshape(1080, 1920, 3)

In [9]:
from PIL import Image
imcpu = Image.fromarray(imgray1)
imcpu.save("/content/drive/MyDrive/Colab Notebooks/image_gray_CPU.jpeg")

In [10]:
cuda.is_available()

True

In [11]:
imflatten2 = im.copy().reshape(shape[0]*shape[1],3)

In [12]:
shape3 = np.shape(imflatten2)

In [13]:
devdata = cuda.to_device(imflatten2)
devOuput = cuda.device_array(shape3, np.uint8)

In [14]:
@cuda.jit
def grayscale(src, dst):
# where are we in the input?
  tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  g = np.uint8((src[tidx, 0] + src[tidx, 1] + src[tidx, 2]) / 3)
  dst[tidx, 0] = dst[tidx, 1] = dst[tidx, 2] = g

In [15]:
t3 = time.time()

pixelCount = shape[0]*shape[1]
blockSize = 64
gridSize = int(pixelCount / blockSize)
grayscale[gridSize, blockSize](devdata, devOuput)

t4 = time.time()

print(t4-t3)


0.15651202201843262


In [16]:
imgray2 = devOuput.copy_to_host()
imgray3 = imgray2.reshape(1080, 1920, 3)
imgpu = Image.fromarray(imgray3)
imgpu.save("/content/drive/MyDrive/Colab Notebooks/image_gray_GPU.jpeg")